# Session 4 - Multi-Head Attention & Fondations du Transformer (CORRECTION)

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Compléter le Multi-Head Attention et comprendre les briques du Transformer

---

## Objectifs pédagogiques

À la fin de cette session, vous serez capable de :
1. Implémenter la classe `MultiHeadAttention` complète
2. Comprendre le **Feed-Forward Network** et son rôle
3. Expliquer **LayerNorm** et les **connexions résiduelles**
4. Implémenter le **masque causal** (différence GPT vs BERT)
5. Avoir une vision claire de l'architecture Transformer

---

## Rappel : Où en sommes-nous ?

Session précédente :
- ✅ Fonction `scaled_dot_product_attention()` complète
- ✅ Classe `SelfAttention` avec W_q, W_k, W_v
- ✅ Visualisation sur CamemBERT
- ✅ Pourquoi Multi-Head + fonction `split_heads()`

Aujourd'hui, on **termine le Multi-Head** et on ajoute les **autres briques** du Transformer.

## 0. Installation et imports

In [ ]:
!pip install torch matplotlib numpy transformers -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math

torch.manual_seed(42)
print(f"PyTorch version: {torch.__version__}")

---

## 1. Rappel : split_heads et attention

Reprenons les fonctions de la session précédente.

In [ ]:
def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Scaled Dot-Product Attention.
    
    Args:
        Q, K, V: shape (..., seq_len, d_k)
        mask: Masque optionnel (True = position masquée)
    
    Returns:
        output, attention_weights
    """
    d_k = K.shape[-1]
    scores = Q @ K.transpose(-2, -1) / math.sqrt(d_k)
    
    if mask is not None:
        scores = scores.masked_fill(mask, float('-inf'))
    
    attention_weights = F.softmax(scores, dim=-1)
    output = attention_weights @ V
    
    return output, attention_weights


def split_heads(x, num_heads):
    """
    Reshape: (batch, seq_len, embed_dim) -> (batch, num_heads, seq_len, d_k)
    """
    batch_size, seq_len, embed_dim = x.shape
    d_k = embed_dim // num_heads
    x = x.view(batch_size, seq_len, num_heads, d_k)
    return x.transpose(1, 2)


print("✅ Fonctions chargées")

---

## 2. Exercice 1 : concat_heads

Après l'attention, on doit **recombiner les têtes**.

C'est l'opération inverse de `split_heads` :
- `(batch, num_heads, seq_len, d_k)` → `(batch, seq_len, embed_dim)`

In [ ]:
def concat_heads(x):
    """
    Recombine les têtes d'attention.
    
    Reshape: (batch, num_heads, seq_len, d_k) -> (batch, seq_len, embed_dim)
    
    C'est l'inverse de split_heads.
    """
    batch_size, num_heads, seq_len, d_k = x.shape
    embed_dim = num_heads * d_k
    
    # CORRECTION:
    # Étape 1: Remettre seq_len en position 1
    x = x.transpose(1, 2)  # -> (batch, seq_len, num_heads, d_k)
    # Étape 2: Fusionner num_heads et d_k
    x = x.contiguous().view(batch_size, seq_len, embed_dim)
    
    return x

In [ ]:
# Test
x_test = torch.randn(2, 4, 6, 8)  # batch=2, heads=4, seq=6, d_k=8
x_concat = concat_heads(x_test)

print(f"Avant concat: {x_test.shape}")   # (2, 4, 6, 8)
print(f"Après concat: {x_concat.shape}") # Attendu: (2, 6, 32)

if x_concat is not None and x_concat.shape == (2, 6, 32):
    print("\n✅ Correct !")
else:
    print("\n❌ Vérifiez votre implémentation")

In [ ]:
# Vérification : split puis concat doit donner le tenseur original
x_original = torch.randn(2, 6, 32)
x_split = split_heads(x_original, num_heads=4)
x_back = concat_heads(x_split)

print(f"Original: {x_original.shape}")
print(f"Split:    {x_split.shape}")
print(f"Concat:   {x_back.shape}")
print(f"\nIdentique ? {torch.allclose(x_original, x_back)}")

---

## 3. Exercice 2 : Classe MultiHeadAttention

Assemblons tout dans une classe `MultiHeadAttention`.

### Architecture

```
x ──► W_q ──► Q ──┐
x ──► W_k ──► K ──┼──► split_heads ──► Attention ──► concat_heads ──► W_o ──► output
x ──► W_v ──► V ──┘
```

In [ ]:
class MultiHeadAttention(nn.Module):
    """
    Multi-Head Attention.
    
    Args:
        embed_dim: Dimension des embeddings
        num_heads: Nombre de têtes d'attention
    """
    
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        
        assert embed_dim % num_heads == 0, "embed_dim doit être divisible par num_heads"
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.d_k = embed_dim // num_heads
        
        # CORRECTION 1: Créer les 4 projections linéaires
        self.W_q = nn.Linear(embed_dim, embed_dim)
        self.W_k = nn.Linear(embed_dim, embed_dim)
        self.W_v = nn.Linear(embed_dim, embed_dim)
        self.W_o = nn.Linear(embed_dim, embed_dim)  # Projection de sortie
    
    def forward(self, x, mask=None):
        """
        Args:
            x: Input, shape (batch, seq_len, embed_dim)
            mask: Masque optionnel
        
        Returns:
            output: (batch, seq_len, embed_dim)
            attention_weights: (batch, num_heads, seq_len, seq_len)
        """
        batch_size, seq_len, _ = x.shape
        
        # CORRECTION 2: Projeter x vers Q, K, V
        Q = self.W_q(x)
        K = self.W_k(x)
        V = self.W_v(x)
        
        # CORRECTION 3: Split en têtes
        Q = split_heads(Q, self.num_heads)
        K = split_heads(K, self.num_heads)
        V = split_heads(V, self.num_heads)
        
        # CORRECTION 4: Appliquer l'attention
        attn_output, attn_weights = scaled_dot_product_attention(Q, K, V, mask)
        
        # CORRECTION 5: Concat des têtes
        concat_output = concat_heads(attn_output)
        
        # CORRECTION 6: Projection de sortie
        output = self.W_o(concat_output)
        
        return output, attn_weights

In [ ]:
# Test
mha = MultiHeadAttention(embed_dim=32, num_heads=4)

x = torch.randn(2, 6, 32)  # batch=2, seq=6, embed=32
output, weights = mha(x)

print(f"Input shape:   {x.shape}")       # (2, 6, 32)
print(f"Output shape:  {output.shape}")  # Attendu: (2, 6, 32)
print(f"Weights shape: {weights.shape}") # Attendu: (2, 4, 6, 6)

# Nombre de paramètres
n_params = sum(p.numel() for p in mha.parameters())
print(f"\nNombre de paramètres: {n_params:,}")

---

## 4. Visualisation : 4 têtes sur CamemBERT

Voyons ce que les différentes têtes capturent sur un modèle **réellement entraîné**.

In [ ]:
from transformers import CamembertModel, CamembertTokenizer

# Charger CamemBERT
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained("camembert-base", output_attentions=True)
model.eval()

# Phrase de test
phrase = "Le chat dort sur le canapé car il est fatigué"
inputs = tokenizer(phrase, return_tensors="pt")
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

with torch.no_grad():
    outputs = model(**inputs)

attentions = outputs.attentions
print(f"Phrase: {phrase}")
print(f"Tokens: {tokens}")

In [ ]:
# Visualiser 4 têtes de la couche 5
layer = 4
heads_to_show = [0, 1, 5, 10]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for idx, head in enumerate(heads_to_show):
    ax = axes[idx // 2, idx % 2]
    w = attentions[layer][0, head].numpy()
    
    im = ax.imshow(w, cmap='Blues')
    ax.set_xticks(range(len(tokens)))
    ax.set_xticklabels(tokens, rotation=45, ha='right', fontsize=8)
    ax.set_yticks(range(len(tokens)))
    ax.set_yticklabels(tokens, fontsize=8)
    ax.set_title(f"Tête {head + 1}", fontsize=11)
    plt.colorbar(im, ax=ax)

plt.suptitle(f"Différentes têtes de la couche {layer+1} - Chaque tête capture des relations différentes", fontsize=12)
plt.tight_layout()
plt.show()

**Observation** : Chaque tête capture des **patterns différents** :
- Certaines regardent les mots **proches**
- D'autres capturent la **coréférence** ("il" → "chat")
- Certaines se concentrent sur **\<s\>** ou **\</s\>**

C'est la force du Multi-Head : **plusieurs experts** qui analysent sous différents angles.

---

## 5. Feed-Forward Network

Après l'attention, chaque position passe par un **réseau feed-forward** identique.

$$\text{FFN}(x) = \text{ReLU}(xW_1 + b_1)W_2 + b_2$$

### Pourquoi un FFN ?

- L'attention capture les **relations entre tokens**
- Le FFN **transforme** chaque token individuellement
- Il ajoute de la **capacité non-linéaire** au modèle

### Dimensions typiques

La dimension cachée est généralement **4× embed_dim** :
- Input: `embed_dim` (ex: 512)
- Hidden: `4 × embed_dim` (ex: 2048)
- Output: `embed_dim` (ex: 512)

In [ ]:
class FeedForward(nn.Module):
    """
    Feed-Forward Network (2 couches linéaires avec activation).
    
    Args:
        embed_dim: Dimension d'entrée/sortie
        ff_dim: Dimension cachée (défaut: 4 × embed_dim)
        dropout: Taux de dropout
    """
    
    def __init__(self, embed_dim, ff_dim=None, dropout=0.1):
        super().__init__()
        
        if ff_dim is None:
            ff_dim = 4 * embed_dim
        
        # CORRECTION: Créer le réseau
        self.linear1 = nn.Linear(embed_dim, ff_dim)
        self.linear2 = nn.Linear(ff_dim, embed_dim)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.GELU()  # GELU est utilisé dans les Transformers modernes
    
    def forward(self, x):
        """
        Args:
            x: shape (batch, seq_len, embed_dim)
        Returns:
            shape (batch, seq_len, embed_dim)
        """
        # CORRECTION: Implémenter le forward
        x = self.linear1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [ ]:
# Test
ff = FeedForward(embed_dim=32, ff_dim=128)
x = torch.randn(2, 6, 32)
out = ff(x)

print(f"Input shape:  {x.shape}")    # (2, 6, 32)
print(f"Output shape: {out.shape}")  # Attendu: (2, 6, 32)

# Paramètres
n_params = sum(p.numel() for p in ff.parameters())
print(f"\nParamètres FFN: {n_params:,}")
print(f"  - linear1: 32×128 + 128 = {32*128 + 128:,}")
print(f"  - linear2: 128×32 + 32 = {128*32 + 32:,}")

---

## 6. LayerNorm et Connexions Résiduelles

### Connexions résiduelles

$$\text{output} = x + \text{SubLayer}(x)$$

**Pourquoi ?** Permet aux gradients de mieux circuler (comme dans ResNet).

### Layer Normalization

Normalise sur la dimension des **features** (pas sur le batch comme BatchNorm).

$$\text{LayerNorm}(x) = \gamma \cdot \frac{x - \mu}{\sigma + \epsilon} + \beta$$

**Pourquoi LayerNorm plutôt que BatchNorm ?**
- Fonctionne avec des **séquences de longueur variable**
- Indépendant de la **taille du batch**

In [ ]:
# Démonstration LayerNorm
x = torch.randn(2, 4, 8)  # (batch, seq, features)

# LayerNorm normalise sur la dernière dimension (features)
ln = nn.LayerNorm(8)
x_norm = ln(x)

print("Avant LayerNorm:")
print(f"  Moyenne par position: {x[0, 0].mean():.4f}")
print(f"  Écart-type par position: {x[0, 0].std():.4f}")

print("\nAprès LayerNorm:")
print(f"  Moyenne par position: {x_norm[0, 0].mean():.4f}")
print(f"  Écart-type par position: {x_norm[0, 0].std():.4f}")

### Architecture d'un bloc Transformer (Pre-LN)

```
Input
  │
  ├──────────────────┐
  ↓                  │ (résiduel)
LayerNorm            │
  ↓                  │
Multi-Head Attention │
  ↓                  │
Dropout              │
  ↓                  │
  + ←────────────────┘
  │
  ├──────────────────┐
  ↓                  │ (résiduel)
LayerNorm            │
  ↓                  │
Feed-Forward         │
  ↓                  │
Dropout              │
  ↓                  │
  + ←────────────────┘
  ↓
Output
```

---

## 7. Masque Causal : GPT vs BERT

### Le problème

Pour **générer du texte** (GPT), le modèle doit prédire le **mot suivant** sans voir le futur.

```
Phrase : "Le chat dort"

BERT (bidirectionnel) :        GPT (causal) :
- "Le" voit tout              - "Le" voit seulement "Le"
- "chat" voit tout            - "chat" voit "Le", "chat"
- "dort" voit tout            - "dort" voit "Le", "chat", "dort"
```

### La solution : Masque causal

On **masque** les positions futures avec $-\infty$ avant le softmax :

$$\text{softmax}(-\infty) = 0$$

In [ ]:
# Créer un masque causal
seq_len = 4

# torch.triu crée une matrice triangulaire supérieure
# diagonal=1 : ne pas inclure la diagonale (un mot peut se voir lui-même)
causal_mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()

print("Masque causal (True = position masquée) :")
print(causal_mask.int())
print()
print("Interprétation :")
print("  Ligne 0 (mot 1) : voit seulement position 0")
print("  Ligne 1 (mot 2) : voit positions 0, 1")
print("  Ligne 2 (mot 3) : voit positions 0, 1, 2")
print("  Ligne 3 (mot 4) : voit tout")

In [ ]:
# Visualisation : attention avec et sans masque
Q = torch.randn(1, 4, 8)
K = torch.randn(1, 4, 8)
V = torch.randn(1, 4, 8)

# Sans masque (BERT-style)
_, weights_bert = scaled_dot_product_attention(Q, K, V, mask=None)

# Avec masque causal (GPT-style)
_, weights_gpt = scaled_dot_product_attention(Q, K, V, mask=causal_mask)

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
tokens_demo = ["Le", "chat", "dort", "bien"]

for idx, (w, title) in enumerate([
    (weights_bert[0], "BERT (bidirectionnel)\nChaque mot voit tout"),
    (weights_gpt[0], "GPT (causal)\nChaque mot voit seulement le passé")
]):
    ax = axes[idx]
    im = ax.imshow(w.numpy(), cmap='Blues')
    ax.set_xticks(range(4))
    ax.set_xticklabels(tokens_demo)
    ax.set_yticks(range(4))
    ax.set_yticklabels(tokens_demo)
    ax.set_xlabel("Tokens regardés")
    ax.set_ylabel("Tokens qui regardent")
    ax.set_title(title)
    plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

### Pourquoi c'est important ?

| Modèle | Masque | Usage |
|--------|--------|-------|
| **BERT** | Pas de masque (bidirectionnel) | Classification, QA, NER |
| **GPT** | Masque causal | Génération de texte |

**Dans le Mini-GPT** (prochaines sessions), on utilisera le masque causal pour générer du texte **caractère par caractère**.

---

## 8. Cross-Attention (mention)

Jusqu'ici, on a fait de la **self-attention** : Q, K, V viennent de la **même source**.

Il existe aussi la **cross-attention** :

```
Self-attention:     x ──► Q, K, V     (même source)

Cross-attention:    x_decoder ──► Q       (une source)
                    x_encoder ──► K, V    (autre source)
```

### Cas d'usage

| Application | Rôle du cross-attention |
|-------------|------------------------|
| **Traduction** | Le décodeur (français) "interroge" l'encodeur (anglais) |
| **RAG** | Le modèle "interroge" les documents récupérés |
| **Image captioning** | Le texte "interroge" l'image |

**On ne l'implémente pas ici**, mais c'est la même mécanique avec Q d'un côté et K, V de l'autre.

---

## 9. Récapitulatif : Les briques du Transformer

### Schéma complet d'un bloc Transformer Encoder

```
┌─────────────────────────────────────────────────────────┐
│                   TRANSFORMER BLOCK                     │
├─────────────────────────────────────────────────────────┤
│                                                         │
│   Input ──────────────────────────────┐                 │
│     │                                 │                 │
│     ↓                                 │                 │
│   LayerNorm                           │                 │
│     │                                 │                 │
│     ↓                                 │                 │
│   Multi-Head Attention                │                 │
│   (+ masque causal si GPT)            │                 │
│     │                                 │                 │
│     ↓                                 │                 │
│   Dropout                             │                 │
│     │                                 │                 │
│     + ←───────────────────────────────┘ (résiduel 1)    │
│     │                                                   │
│     ├──────────────────────────────┐                    │
│     ↓                              │                    │
│   LayerNorm                        │                    │
│     │                              │                    │
│     ↓                              │                    │
│   Feed-Forward (expand 4×)         │                    │
│     │                              │                    │
│     ↓                              │                    │
│   Dropout                          │                    │
│     │                              │                    │
│     + ←────────────────────────────┘ (résiduel 2)       │
│     │                                                   │
│     ↓                                                   │
│   Output                                                │
│                                                         │
└─────────────────────────────────────────────────────────┘
```

### Tableau récapitulatif

| Composant | Rôle | Paramètres |
|-----------|------|------------|
| **Multi-Head Attention** | Capturer les relations entre tokens | W_q, W_k, W_v, W_o |
| **Feed-Forward** | Transformation non-linéaire par position | W_1, b_1, W_2, b_2 |
| **LayerNorm** | Stabiliser l'entraînement | γ, β |
| **Résiduel** | Faciliter le flux des gradients | - |
| **Masque causal** | Empêcher de voir le futur (GPT) | - |

### Prochaine session

On va :
1. **Assembler** un bloc Transformer complet
2. Commencer le **Mini-GPT** : générer des noms de Pokémon !

---

## 10. Pour aller plus loin (optionnel)

### Comparaison avec nn.MultiheadAttention de PyTorch

In [ ]:
# PyTorch fournit une implémentation optimisée
mha_pytorch = nn.MultiheadAttention(embed_dim=32, num_heads=4, batch_first=True)

x = torch.randn(2, 6, 32)

# Pour nn.MultiheadAttention, on passe query, key, value séparément
# En self-attention, query = key = value = x
output_pytorch, weights_pytorch = mha_pytorch(x, x, x)

print(f"Output shape (PyTorch): {output_pytorch.shape}")
print(f"Weights shape (PyTorch): {weights_pytorch.shape}")

# Avec masque causal
causal_mask_pytorch = nn.Transformer.generate_square_subsequent_mask(6)
output_causal, _ = mha_pytorch(x, x, x, attn_mask=causal_mask_pytorch)
print(f"\nAvec masque causal: {output_causal.shape}")

### Calcul du nombre de paramètres

In [ ]:
# Exercice : calculer les paramètres d'un bloc Transformer
embed_dim = 512
num_heads = 8
ff_dim = 2048  # 4 × embed_dim

# Multi-Head Attention : 4 matrices (W_q, W_k, W_v, W_o)
params_mha = 4 * (embed_dim * embed_dim + embed_dim)  # weights + bias

# Feed-Forward : 2 couches
params_ff = (embed_dim * ff_dim + ff_dim) + (ff_dim * embed_dim + embed_dim)

# LayerNorm : 2 × (gamma + beta)
params_ln = 2 * (embed_dim + embed_dim)

total = params_mha + params_ff + params_ln

print(f"Paramètres par bloc Transformer (embed_dim={embed_dim}):")
print(f"  Multi-Head Attention: {params_mha:,}")
print(f"  Feed-Forward:         {params_ff:,}")
print(f"  LayerNorm (×2):       {params_ln:,}")
print(f"  ─────────────────────────────")
print(f"  Total:                {total:,}")
print(f"\nPour 12 blocs (BERT): {total * 12:,} ≈ {total * 12 / 1e6:.0f}M paramètres")